[LangChain の QAGenerationChain によるドキュメントからのQA生成を試す
](https://note.com/npaka/n/ncf1dbb190caf)

In [1]:
# https://www.nintendo.co.jp/ir/pdf/2023/security_q2303.pdf
!curl -o ../data/nintendo.pdf https://www.nintendo.co.jp/ir/pdf/2023/security_q2303.pdf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1278k  100 1278k    0     0  5483k      0 --:--:-- --:--:-- --:--:-- 5464k


In [2]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("../data/nintendo.pdf")

In [3]:
documents = loader.load_and_split()

In [4]:
import getpass

OPENAI_MODEL_NAME = "gpt-35-turbo"
OPENAI_DEPLOYMENT_ID = "chat"
OPENAI_API_TYPE = "azure"
OPENAI_API_VERSION = "2023-09-01-preview"

# "https://<your-endpoint.openai.azure.com/"
OPENAI_API_BASE = input('OpenAI API Base:')
OPENAI_API_KEY = getpass.getpass('OpenAI API Key:')

OpenAI API Base: https://aoai-r7a5whblfnou2.openai.azure.com/
OpenAI API Key: ········


In [5]:
# Chat model
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    model_name=OPENAI_MODEL_NAME,
    openai_api_base=OPENAI_API_BASE,
    openai_api_key=OPENAI_API_KEY,
    temperature=0,
    model_kwargs={
        "deployment_id": OPENAI_DEPLOYMENT_ID,
        "api_type": OPENAI_API_TYPE,
        "api_version": OPENAI_API_VERSION,
    },
)

In [6]:
from langchain.chains import QAGenerationChain

chain = QAGenerationChain.from_llm(
    llm=llm,
)
chain

QAGenerationChain(llm_chain=LLMChain(prompt=ChatPromptTemplate(input_variables=['text'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a smart assistant designed to help high school teachers come up with reading comprehension questions.\nGiven a piece of text, you must come up with a question and answer pair that can be used to test a student\'s reading comprehension abilities.\nWhen coming up with this question/answer pair, you must respond in the following format:\n```\n{{\n    "question": "$YOUR_QUESTION_HERE",\n    "answer": "$THE_ANSWER_HERE"\n}}\n```\n\nEverything between the ``` must be valid json.\n')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='Please come up with a question/answer pair, in the specified JSON format, for the following text:\n----------------\n{text}'))]), llm=ChatOpenAI(client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-35-turbo', te

In [7]:
from langchain.prompts.chat import (
    PromptTemplate,
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

system_message = 'You are a smart assistant designed to help high school teachers come up with reading comprehension questions.\nGiven a piece of text, you must come up with a question and answer pair that can be used to test a student\'s reading comprehension abilities.\nWhen coming up with this question/answer pair, you must respond in the following format:\n```\n{{\n    "question": "$YOUR_QUESTION_HERE",\n    "answer": "$THE_ANSWER_HERE"\n}}\n```\n\nEverything between the ``` must be valid json.\n'

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate(
        prompt=PromptTemplate(
            input_variables=[],
            template=system_message + 'answer in Japanese',
        )
    ),
    HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            input_variables=['text'],
            template='Please come up with a question/answer pair, in the specified JSON format, for the following text:\n----------------\n{text}',
        )
    )
])

chain = QAGenerationChain.from_llm(
    llm=llm,
    prompt=prompt,
)

In [8]:
count_max = 10
count = 0
for document in documents:
    print(chain.run(document.page_content))
    count = count + 1
    if count >= count_max:
        break

[{'question': 'この文書は何の目的で提出されましたか？', 'answer': '有価証券報告書を提出するため'}]
[{'question': '2023年3月の売上高はいくらですか？', 'answer': '1,601,677百万円'}]
[{'question': '2023年3月の売上高は何百万円ですか？', 'answer': '1,409,503'}]
[{'question': '任天堂はいつ株式会社丸福として発足しましたか？', 'answer': '1947年11月'}]
[{'question': '当社及び関係会社が事業として行っている主な活動は何ですか？', 'answer': 'ホームエンターテインメントの分野で娯楽製品の開発、製造及び販売等を事業としています。'}]
[{'question': '事業系統図は何を示すために使用されますか？', 'answer': '事業系統図は前述の事項を示すために使用されます。'}]
[{'question': '任天堂の関係会社の中で、最も多くの従業員を抱えているのはどこですか？', 'answer': 'NintendoofAmericaInc.'}]
[{'question': '上記のテキストによれば、連結子会社の数はいくつですか？', 'answer': '5社'}]
[{'question': '提出会社の従業員数は何人ですか？', 'answer': '2,779人'}]
[{'question': '当社グループの経営方針は何ですか？', 'answer': '当社グループの経営方針は、「娯楽を通じて人々を笑顔にする会社」として、健全な企業経営を維持しつつ新しい娯楽の創造を目指すことです。'}]
